In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

## Redis Hashes Explained - [Link to Video](https://youtu.be/-agsJUihrWw)

### Overview

Redis Hashes are collections of field–value pairs stored under a single key, suitable for modeling objects and grouping related counters. Fields and values are strings; hashes are flat (no nested arrays or objects) and schemaless. They support constant-time access and updates to individual fields. See: [Redis hashes](https://redis.io/docs/latest/develop/data-types/hashes/), [Hash performance & limits](https://redis.io/docs/latest/develop/data-types/hashes/#performance).

> **Sidenote**  
>
> [Link: https://redis.io/docs/latest/develop/data-types/hashes/](https://redis.io/docs/latest/develop/data-types/hashes/)  
>
> **Concept**: `Redis Hash` → A record type of string field–value pairs.  
>
> **Context**: Use to represent lightweight objects (e.g., a user, product, or game player) or bundles of counters.  
>
> **Example**: `player:42` holding `name`, `race`, `level`, `hp`, `gold`.  
>
> **Implication**:  
>
> O(1) access for most hash commands; O(n) for enumeration commands like `HGETALL`. Hashes are flat (strings only), so nested structures require other types (e.g., RedisJSON).

### Key Modeling and Naming

Use namespaced keys with `:` as a delimiter (e.g., `player:42`). This convention clarifies type and identity and composes well with features like hash tags for clustering. See: [Keys & values](https://redis.io/docs/latest/develop/using-commands/keyspace/), [Redis namespace best practice (blog)](https://redis.io/blog/5-key-takeaways-for-developing-with-redis/).

> **Sidenote**  
>
> [Link: https://redis.io/blog/5-key-takeaways-for-developing-with-redis/](https://redis.io/blog/5-key-takeaways-for-developing-with-redis/)  
>
> **Concept**: `Key Namespacing` → Delimit parts with `:` (e.g., `objectType:id`).  
>
> **Context**: Organizes data, improves readability, and simplifies migrations/cleanup.  
>
> **Example**: `player:42`, `player:42:stats`.  
>
> **Implication**:  
>
> Predictable key layouts and easier operations in clustered deployments (with hashtags if needed).

### Create a Player Hash (`HSET`)

We model the Mages & Minotaurs player **Artexius** as a Redis Hash. Initial fields: `name`, `race`, `level`, `hp` (health points), and `gold`. `HSET` creates the hash if it doesn’t exist and returns the count of fields **added** (not overwritten). See: [HSET](https://redis.io/docs/latest/commands/hset/).

```python
# create player:42 with initial attributes
r.execute_command(
    "HSET", "player:42",
    "name", "Artexius",
    "race", "Elf",
    "level", "5",
    "hp", "72",
    "gold", "50"
)
print(r.execute_command("HLEN", "player:42"))  # verification: number of fields, expect 5
````

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/hset/](https://redis.io/docs/latest/commands/hset/)
>
> **Command**: `HSET` → Set one or more fields in a hash (creates hash if missing; overwrites existing fields).
>
> **Pattern**: `HSET key field value [field value ...]`
>
> **Example**: `HSET player:42 level 5 hp 72`
>
> **Result**:
>
> Integer: number of fields **added** (overwrites don’t count).

### Add a Temporary Status Field (Update with `HSET`)

We add a temporary `status` field (e.g., `dazed`) during gameplay. Same command; overwrites if the field already exists.

```python
# set temporary status field
r.execute_command("HSET", "player:42", "status", "dazed")
print(r.execute_command("HGET", "player:42", "status"))  # verification: expect "dazed"
```

### Remove a Field (`HDEL`)

When the temporary status no longer applies, delete it with `HDEL`. Returns the count of fields removed. See: [HDEL](https://redis.io/docs/latest/commands/hdel/).

```python
# remove status when effect ends
r.execute_command("HDEL", "player:42", "status")
print(r.execute_command("HEXISTS", "player:42", "status"))  # verification: expect 0 (false)
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/hdel/](https://redis.io/docs/latest/commands/hdel/)
>
> **Command**: `HDEL` → Delete one or more fields from a hash.
>
> **Pattern**: `HDEL key field [field ...]`
>
> **Example**: `HDEL player:42 status`
>
> **Result**:
>
> Integer: number of fields actually removed.

### Read a Single Field (`HGET`)

Retrieve a specific attribute (e.g., `level`). Returns the stored string or a null reply if absent. See: [HGET](https://redis.io/docs/latest/commands/hget/).

```python
# read one field
r.execute_command("HGET", "player:42", "level")
print(r.execute_command("HGET", "player:42", "level"))  # verification: expect "5"
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/hget/](https://redis.io/docs/latest/commands/hget/)
>
> **Command**: `HGET` → Get the value of a hash field.
>
> **Pattern**: `HGET key field`
>
> **Example**: `HGET player:42 level`
>
> **Result**:
>
> Bulk string: field value, or null if the field does not exist.

### Read All Fields (`HGETALL`) — Use with Care on Large Hashes

Fetch the entire object with `HGETALL`. Complexity is O(n) in the number of fields; for large hashes, prefer targeted reads or incremental scans. See: [HGETALL](https://redis.io/docs/latest/commands/hgetall/).

```python
# read the whole object
r.execute_command("HGETALL", "player:42")
print(r.execute_command("HGETALL", "player:42"))  # verification: flat list of alternating field and value
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/hgetall/](https://redis.io/docs/latest/commands/hgetall/)
>
> **Command**: `HGETALL` → Get all fields and values from a hash.
>
> **Pattern**: `HGETALL key`
>
> **Example**: `HGETALL player:42`
>
> **Result**:
>
> Array of alternating field and value entries. O(n) complexity with respect to field count.

### Increment Numeric Fields (`HINCRBY`)

Increase or decrease a numeric field atomically. If the hash or field doesn’t exist, they are created with `0` before the increment. The value range is 64-bit signed integer. See: [HINCRBY](https://redis.io/docs/latest/commands/hincrby/).

```python
# add gold after a quest (+120)
r.execute_command("HINCRBY", "player:42", "gold", "120")
print(r.execute_command("HGET", "player:42", "gold"))  # verification: expect "170" (50 + 120)
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/hincrby/](https://redis.io/docs/latest/commands/hincrby/)
>
> **Command**: `HINCRBY` → Atomically add an integer delta to a hash field.
>
> **Pattern**: `HINCRBY key field increment`
>
> **Example**: `HINCRBY player:42 gold 120`
>
> **Result**:
>
> Integer: updated value after the increment. Creates the hash/field if absent; supports negative increments for decrements; 64-bit signed integer range.

### Scanning Large Hashes (`HSCAN`)

For large hashes, iterate incrementally with a cursor using `HSCAN` (supports `MATCH` and `COUNT`, and `NOVALUES` from Redis 7.4). Each call is O(1); a full iteration is O(n). See: [HSCAN](https://redis.io/docs/latest/commands/hscan/).

```python
# iterate fields lazily (pattern match optional)
cursor = "0"
while True:
    cursor, items = r.execute_command("HSCAN", "player:42", cursor, "MATCH", "*")
    # items is [field1, value1, field2, value2, ...]
    if cursor == "0":
        break
print(len(items) % 2 == 0)  # verification: even length list of field/value pairs
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/commands/hscan/](https://redis.io/docs/latest/commands/hscan/)
>
> **Command**: `HSCAN` → Incrementally iterate fields and values in a hash.
>
> **Pattern**: `HSCAN key cursor [MATCH pattern] [COUNT count] [NOVALUES]`
>
> **Example**: `HSCAN player:42 0 MATCH hp* COUNT 100`
>
> **Result**:
>
> Two-element array: next `cursor` and an array of field/value pairs (or just fields with `NOVALUES`).

### Performance Characteristics and Limits

Most hash commands (`HSET`, `HGET`, `HDEL`, `HINCRBY`, etc.) are O(1). Enumeration commands (`HGETALL`, `HKEYS`, `HVALS`) are O(n). Each hash can store up to **4,294,967,295 (2^32 − 1)** field–value pairs (practically limited by memory). See: [Hash performance & limits](https://redis.io/docs/latest/develop/data-types/hashes/#performance).

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/develop/data-types/hashes/#performance](https://redis.io/docs/latest/develop/data-types/hashes/#performance)
>
> **Concept**: `Time Complexity & Limits` → O(1) for most single-field ops; O(n) for full enumerations; theoretical per-hash field limit is 2^32−1.
>
> **Context**: Choose targeted reads (`HGET`, `HMGET`) and incremental scans for large objects.
>
> **Example**: Prefer `HGET player:42 hp` over `HGETALL player:42` in hot paths.
>
> **Implication**:
>
> Predictable latency for point lookups; avoid O(n) calls in critical paths.

### Flat vs. Nested Structures; When to Use RedisJSON

Hashes are flat—values are strings, so you cannot nest arrays/objects inside a hash. For hierarchical/nested JSON documents and JSON-path updates/queries, use RedisJSON. See: [Redis hashes](https://redis.io/docs/latest/develop/data-types/hashes/), [Redis JSON](https://redis.io/docs/latest/develop/data-types/json/).

```python
# store nested data? Use RedisJSON (module) instead of encoding JSON into multiple hash fields
# example (JSON CLI shown for illustration; Python clients provide JSON helpers):
# JSON.SET player:42 $ '{"name":"Artexius","race":"Elf","stats":{"level":5,"hp":72},"gold":50}'
print("Use RedisJSON for nested structures and JSONPath operations")
```

> **Sidenote**
>
> [Link: https://redis.io/docs/latest/develop/data-types/json/](https://redis.io/docs/latest/develop/data-types/json/)
>
> **Concept**: `RedisJSON` → Native JSON data type with efficient storage, updates, and querying.
>
> **Context**: Use when object graphs are nested or when you need JSONPath reads/writes and indexing.
>
> **Example**: `JSON.SET player:42 $ '{"stats":{"hp":72}}'`
>
> **Implication**:
>
> Enables hierarchical modeling and rich querying beyond flat hash capabilities.



### Original Transcript

Video title: Redis Hashes Explained
    
Video URL: https://youtu.be/-agsJUihrWw
    
Video language: English
    
--------------------------------

Hello. Today, we're going to learn about Hashes, an especially useful Redis data structure. In this segment, I'll explain what Hashes are, and I'll show you how to use them to model domain objects in an application.  Hashes are collections of field-value pairs and look a lot like a JSON Object, a Java HashMap, or a Python Dictionary. Redis Hashes are also mutable. We can add, change, increment, and remove field-value pairs at any time, not just at the initial declaration. Hashes store field values as Strings, which means that they are flat. There are no nested Arrays or Objects. Also, we do not need to predefine field names of Redis Hashes. And as such, we can add and remove fields as needed. While Redis Hashes are schemaless, you can still think of them as lightweight objects or as rows in a relational database table. To explore Redis Hashes in practice, let's think about how we'd model a player in a fictional online role-playing game, Mages & Minotaurs. [THEME MUSIC PLAYING]  In a role-playing game, players live, die, and are reborn, and this cycle of life is reflected in health points, armor, abilities, and battles. We'll store this information in a Hash with every player getting their own Hash instance. We'll update this player Hash with new information, create temporary fields, and use built-in functionality to increment numerical values and update strings. Enter the player Artexius, the proud Elven knight. For this first example, we'll create a Hash starting off with the field's name, race, level, health points, and gold. To create this Hash, we'll use the command HSET. The first argument to the HSET command is the key name we'll use to access the Hash. In this case, it's player:42. Here, we're using a common Redis key naming convention. We start with the word "player" to indicate what type of thing we're storing. We then followed with a colon and the ID of this player. The colon separates the various parts of the key name, going from least specific on the left to most specific on the right. After specifying the key, we add as many field-value pairs as we want. When we run this command, Redis returns 5, indicating the number of fields saved to the Hash. We've now created a Redis Hash, our first player for the Mages & Minotaurs. Now, let's see how to update and delete fields within a Redis Hash. Imagine the player Artexius is having an epic battle with a wizard and, ooh, received a Thunderbolt spell to the head. In the game, he'll have a status of "dazed." To reflect this game state in the Redis Hash, we will add the field-value pair "status: dazed" to the player Hash instance. Here again, we'll use the HSET command to add a new field-value pair. So the command will run as HSET player:42 status dazed. What happens when our player no longer has the status of "dazed"? We'll want to remove the status field from the Hash instance. To delete a field-value pair from a Redis Hash, we use the command HDEL. The command is HDEL player:42 status. Now, the player:42 Hash instance no longer has a "dazed" status. And Artexius lives to quest another day. Getting data from a Hash is just as easy as setting it. Suppose we need to retrieve Artexius's level. For that, we'll need to use the HGET command. The command is HGET player:42 and then the field we want, "level." To get all the fields and values from a Redis Hash, use the HGETALL command. We'll run HGETALL player:42. This will return all fields and values contained within the Hash. Now, in many role-playing games, players receive gold after completing objectives or defeating enemies. We'll need to be able to increase the amount of gold a player has whenever this occurs. Guess what? There's a command for that. It's called HINCRBY, and it increments the value of a particular Hash field. We enter HINCRBY player:42 gold, and the increment value. Let's increment the value assigned to the gold field by 120. OK. Finally, let's talk about the performance characteristics of these Hash commands. Most Hash commands are O(1) , which means they will always perform a task in a constant amount of time, regardless of the size of the Hash. Constant time commands are as efficient as it gets. The HGETALL command is O(n) , with n being the number of fields within a Hash. This means that the amount of time for the task to complete is dependent on how many fields it has to retrieve. In this case, the n of O(n) is the number of fields that the Hash contains. HGETALL is practical for relatively small Hashes, such as our player Hash. For Hashes containing thousands of fields or more, it's usually most efficient to select the exact fields you want using HGET rather than retrieving all of the data with HGETALL or HSCAN. OK. Let's review. We've just explored how we can use Redis Hashes to represent player objects in an online role-playing game. We've learned that Redis Hashes exist as collections of field-value pairs. We created and updated Hashes using HSET. We retrieved data with HGET and HGETALL. Fields can be deleted with HDEL. Lastly, we incremented numerical values using HINCRBY. As useful as Hashes are, they aren't the be-all and end-all for object storage in Redis. If you're dealing with nested structures or JSON, check out RedisJSON. RedisJSON is a Redis module that implements the JSON data interchange standard as a native data type. It implements efficient storage, update, and retrieval of JSON data in Redis. To learn more about Redis Hashes, check our free online course: Introduction to Redis Data Structures. It's part of Redis University, our online learning platform for all things Redis. Thank you for joining me in designing Mages & Minotaurs. I hope to go on another quest with you again soon. [MUSIC PLAYING]